In [21]:
import pandas as pd 
import numpy as np
import torch
import os
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import relieff

In [22]:
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/NSW2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data.resample('D').mean()], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('NSW_all.csv')
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/NSW2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('NSW_all_hour.csv')


                      RRP
SETTLEMENTDATE           
2009-01-01      23.505745
2009-01-02      19.446250
2009-01-03      19.460625
2009-01-04      19.064792
2009-01-05      36.229167
...                   ...
2019-12-28      49.402500
2019-12-29      53.891042
2019-12-30      56.572708
2019-12-31      49.408750
2020-01-01      50.010000

[4059 rows x 1 columns]
                       RRP
SETTLEMENTDATE            
2009-01-01 00:30:00  18.38
2009-01-01 01:00:00  19.67
2009-01-01 01:30:00  16.61
2009-01-01 02:00:00  14.57
2009-01-01 02:30:00  13.87
...                    ...
2019-12-31 22:00:00  49.01
2019-12-31 22:30:00  55.25
2019-12-31 23:00:00  50.64
2019-12-31 23:30:00  49.98
2020-01-01 00:00:00  50.01

[195792 rows x 1 columns]


In [23]:
import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/NSW2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"].resample('D').mean()
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('NSW.csv', index=False)
import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/NSW2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"]
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('NSW_hour.csv', index=False)


SETTLEMENTDATE
2018-01-01    74.691277
2018-01-02    72.193958
2018-01-03    65.044167
2018-01-04    63.988333
2018-01-05    65.690833
                ...    
2019-12-28    49.402500
2019-12-29    53.891042
2019-12-30    56.572708
2019-12-31    49.408750
2020-01-01    50.010000
Freq: D, Name: RRP, Length: 731, dtype: float64
SETTLEMENTDATE
2018-01-01 00:30:00    91.86
2018-01-01 01:00:00    88.83
2018-01-01 01:30:00    73.62
2018-01-01 02:00:00    71.49
2018-01-01 02:30:00    69.27
                       ...  
2019-12-31 22:00:00    49.01
2019-12-31 22:30:00    55.25
2019-12-31 23:00:00    50.64
2019-12-31 23:30:00    49.98
2020-01-01 00:00:00    50.01
Name: RRP, Length: 35040, dtype: float64


In [24]:
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/QLD2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data.resample('D').mean()], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('QLD_all.csv')
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/QLD2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('QLD_all_hour.csv')


                      RRP
SETTLEMENTDATE           
2009-01-01      31.776809
2009-01-02      18.792292
2009-01-03      18.307083
2009-01-04      17.148542
2009-01-05      30.508958
...                   ...
2019-12-28      46.409375
2019-12-29      50.558333
2019-12-30      54.476667
2019-12-31      48.233125
2020-01-01      52.150000

[4028 rows x 1 columns]
                       RRP
SETTLEMENTDATE            
2009-01-01 00:30:00  17.63
2009-01-01 01:00:00  18.73
2009-01-01 01:30:00  15.43
2009-01-01 02:00:00  13.25
2009-01-01 02:30:00  12.97
...                    ...
2019-12-31 22:00:00  50.01
2019-12-31 22:30:00  56.84
2019-12-31 23:00:00  52.52
2019-12-31 23:30:00  51.70
2020-01-01 00:00:00  52.15

[192816 rows x 1 columns]


In [25]:
import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/QLD2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"].resample('D').mean()
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('QLD.csv', index=False)
import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/QLD2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"]
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('QLD_hour.csv', index=False)


SETTLEMENTDATE
2018-01-01    72.750638
2018-01-02    74.462500
2018-01-03    65.422083
2018-01-04    65.145208
2018-01-05    65.574792
                ...    
2019-12-28    46.409375
2019-12-29    50.558333
2019-12-30    54.476667
2019-12-31    48.233125
2020-01-01    52.150000
Freq: D, Name: RRP, Length: 731, dtype: float64
SETTLEMENTDATE
2018-01-01 00:30:00    81.27
2018-01-01 01:00:00    78.02
2018-01-01 01:30:00    65.02
2018-01-01 02:00:00    64.85
2018-01-01 02:30:00    64.23
                       ...  
2019-12-31 22:00:00    50.01
2019-12-31 22:30:00    56.84
2019-12-31 23:00:00    52.52
2019-12-31 23:30:00    51.70
2020-01-01 00:00:00    52.15
Name: RRP, Length: 35040, dtype: float64


In [26]:
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/SA2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data.resample('D').mean()], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('SA_all.csv')
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/SA2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('SA_all_hour.csv')


                       RRP
SETTLEMENTDATE            
2009-01-01       14.907872
2009-01-02       17.764167
2009-01-03       20.332708
2009-01-04       21.327708
2009-01-05       35.105208
...                    ...
2019-12-28       68.081250
2019-12-29       87.934167
2019-12-30      124.902500
2019-12-31       45.509167
2020-01-01       85.950000

[4028 rows x 1 columns]
                       RRP
SETTLEMENTDATE            
2009-01-01 00:30:00  21.87
2009-01-01 01:00:00  20.34
2009-01-01 01:30:00  16.81
2009-01-01 02:00:00  13.71
2009-01-01 02:30:00  12.93
...                    ...
2019-12-31 22:00:00  57.12
2019-12-31 22:30:00  61.83
2019-12-31 23:00:00  57.42
2019-12-31 23:30:00  68.58
2020-01-01 00:00:00  85.95

[192816 rows x 1 columns]


In [27]:
import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/SA2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"].resample('D').mean()
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('SA.csv', index=False)

import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/SA2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"]
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('SA_hour.csv', index=False)



SETTLEMENTDATE
2018-01-01     77.461915
2018-01-02     69.336458
2018-01-03     70.328958
2018-01-04     83.527292
2018-01-05    102.623958
                 ...    
2019-12-28     68.081250
2019-12-29     87.934167
2019-12-30    124.902500
2019-12-31     45.509167
2020-01-01     85.950000
Freq: D, Name: RRP, Length: 731, dtype: float64
SETTLEMENTDATE
2018-01-01 00:30:00    107.17
2018-01-01 01:00:00    103.31
2018-01-01 01:30:00     88.20
2018-01-01 02:00:00     85.24
2018-01-01 02:30:00     81.75
                        ...  
2019-12-31 22:00:00     57.12
2019-12-31 22:30:00     61.83
2019-12-31 23:00:00     57.42
2019-12-31 23:30:00     68.58
2020-01-01 00:00:00     85.95
Name: RRP, Length: 35040, dtype: float64


In [28]:
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/TAS2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data.resample('D').mean()], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('TAS_all.csv')
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/TAS2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('TAS_all_hour.csv')


                      RRP
SETTLEMENTDATE           
2009-01-01      25.613404
2009-01-02      28.231667
2009-01-03      28.533958
2009-01-04      32.110208
2009-01-05      50.105000
...                   ...
2019-12-28      78.448750
2019-12-29      80.500833
2019-12-30      73.376458
2019-12-31      65.744792
2020-01-01      93.880000

[4028 rows x 1 columns]
                       RRP
SETTLEMENTDATE            
2009-01-01 00:30:00  25.20
2009-01-01 01:00:00  17.20
2009-01-01 01:30:00   1.65
2009-01-01 02:00:00   7.14
2009-01-01 02:30:00  25.25
...                    ...
2019-12-31 22:00:00  65.31
2019-12-31 22:30:00  80.97
2019-12-31 23:00:00  73.03
2019-12-31 23:30:00  81.76
2020-01-01 00:00:00  93.88

[192816 rows x 1 columns]


In [29]:
import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/TAS2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and file.startswith('PRICE')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"].resample('D').mean()
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('TAS.csv', index=False)

import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/TAS2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and file.startswith('PRICE')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"]
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('TAS_hour.csv', index=False)



SETTLEMENTDATE
2018-01-01    77.555319
2018-01-02    76.905417
2018-01-03    80.044375
2018-01-04    84.265000
2018-01-05    90.936875
                ...    
2019-12-28    78.448750
2019-12-29    80.500833
2019-12-30    73.376458
2019-12-31    65.744792
2020-01-01    93.880000
Freq: D, Name: RRP, Length: 731, dtype: float64
SETTLEMENTDATE
2018-01-01 00:30:00    92.28
2018-01-01 01:00:00    87.53
2018-01-01 01:30:00    76.29
2018-01-01 02:00:00    75.10
2018-01-01 02:30:00    72.92
                       ...  
2019-12-31 22:00:00    65.31
2019-12-31 22:30:00    80.97
2019-12-31 23:00:00    73.03
2019-12-31 23:30:00    81.76
2020-01-01 00:00:00    93.88
Name: RRP, Length: 35040, dtype: float64


In [30]:
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/VIC2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data.resample('D').mean()], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('VIC_all.csv')
import os
import pandas as pd

# 主文件夹路径
main_folder_path = '../data/澳洲电力负荷数据/VIC2009-2019'

# 获取主文件夹下的所有子文件夹（每个子文件夹表示当年的数据）
yearly_folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]

# 初始化一个空的 DataFrame，用于存储合并后的数据
combined_data = pd.DataFrame()

# 遍历每个子文件夹
for year_folder in yearly_folders:
    year_folder_path = os.path.join(main_folder_path, year_folder)
    
    # 获取子文件夹下的所有 CSV 文件
    csv_files = [f for f in os.listdir(year_folder_path) if f.endswith('.csv')]
    
    # 读取并合并每个月的数据
    monthly_data = pd.concat([pd.read_csv(os.path.join(year_folder_path, file)) for file in csv_files], ignore_index=True)
    
    # 将时间列转换为 Pandas 的日期时间类型
    monthly_data['SETTLEMENTDATE'] = pd.to_datetime(monthly_data['SETTLEMENTDATE'])  # 替换 'timestamp' 为你的时间列名称
    
    # 将时间列设置为索引
    monthly_data.set_index('SETTLEMENTDATE', inplace=True)
    monthly_data = monthly_data[["RRP"]]
    
    # 将数据按日进行采样并添加到合并后的数据中
    combined_data = pd.concat([combined_data, monthly_data], ignore_index=False)

# 按照时间排序
combined_data.sort_index(inplace=True)

# 打印合并、按日采样和排序后的数据
print(combined_data)

# 将合并后的数据写入新的 CSV 文件
combined_data.to_csv('VIC_all_hour.csv')


                       RRP
SETTLEMENTDATE            
2009-01-01       16.577660
2009-01-02       16.771667
2009-01-03       17.246875
2009-01-04       17.321875
2009-01-05       32.792708
...                    ...
2019-12-28       62.217292
2019-12-29       82.885000
2019-12-30      235.758333
2019-12-31       45.588958
2020-01-01       85.230000

[4028 rows x 1 columns]
                       RRP
SETTLEMENTDATE            
2009-01-01 00:30:00  16.76
2009-01-01 01:00:00  15.34
2009-01-01 01:30:00  14.93
2009-01-01 02:00:00  10.97
2009-01-01 02:30:00  11.86
...                    ...
2019-12-31 22:00:00  52.91
2019-12-31 22:30:00  58.66
2019-12-31 23:00:00  54.36
2019-12-31 23:30:00  66.87
2020-01-01 00:00:00  85.23

[192816 rows x 1 columns]


In [31]:
import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/VIC2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"].resample('D').mean()
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('VIC.csv', index=False)
import os
import pandas as pd

# 指定文件夹路径
folder_path = '../data/澳洲电力负荷数据/VIC2018-2019'

# 获取文件夹中所有 CSV 文件的文件名
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并拼接所有 CSV 文件
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
# 按照时间列排序

# 查找缺失秒信息的数据
missing_seconds = combined_df['SETTLEMENTDATE'].str.count(':') != 2  # 判断是否缺失秒信息

# 补全缺失秒信息的数据，将其秒部分补全为 0
combined_df.loc[missing_seconds, 'SETTLEMENTDATE'] += ":00"
# print(combined_df)

combined_df['SETTLEMENTDATE'] = pd.to_datetime(combined_df['SETTLEMENTDATE'])  # 替换 'your_time_column' 为你的时间列名称
combined_df = combined_df.sort_values(by='SETTLEMENTDATE')

# 如果需要重置索引
# combined_df.reset_index(drop=True, inplace=True)
combined_df.index = combined_df['SETTLEMENTDATE']
combined_df = combined_df["RRP"]
# 打印合并后的 DataFrame
print(combined_df)

# 将合并后的 DataFrame 写入新的 CSV 文件
combined_df.to_csv('VIC_hour.csv', index=False)


SETTLEMENTDATE
2018-01-01     71.063404
2018-01-02     68.788125
2018-01-03     66.658333
2018-01-04     77.299583
2018-01-05     88.099167
                 ...    
2019-12-28     62.217292
2019-12-29     82.885000
2019-12-30    235.758333
2019-12-31     45.588958
2020-01-01     85.230000
Freq: D, Name: RRP, Length: 731, dtype: float64
SETTLEMENTDATE
2018-01-01 00:30:00    92.46
2018-01-01 01:00:00    87.62
2018-01-01 01:30:00    73.08
2018-01-01 02:00:00    70.18
2018-01-01 02:30:00    67.43
                       ...  
2019-12-31 22:00:00    52.91
2019-12-31 22:30:00    58.66
2019-12-31 23:00:00    54.36
2019-12-31 23:30:00    66.87
2020-01-01 00:00:00    85.23
Name: RRP, Length: 35040, dtype: float64
